In [1]:
import warnings
warnings.filterwarnings("always")
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np


In [2]:
df=pd.read_csv("X_full.csv",index_col=0)
df_raw=pd.read_csv("F:/Minor project dataset/Datasets/LoanExport.csv",index_col=0)

In [3]:
df.head()

,CreditScore,FirstTimeHomebuyer,MSA,MIP,Units,Occupancy,OCLTV,DTI,OrigUPB,LTV,...,FirstPaymentYear,FirstPaymentMonth,Maturity_Year,Maturity_Month,LTV_range,Credit_range,YearsInRepayment,Repay_range,IsFirstTimeHomeBuyer,Duration
0,711.841009,0,16974,25,1,0,89,27.0,117000,89.0,...,1999,2,2029,1,0,0,4.333333,2,0,30
1,711.841009,0,19740,0,1,0,73,17.0,109000,73.0,...,1999,2,2029,1,0,0,12.000000,4,0,30
2,711.841009,0,29940,0,1,0,75,16.0,88000,75.0,...,1999,2,2029,1,0,0,5.583333,2,0,30
3,711.841009,0,31084,0,1,0,76,14.0,160000,76.0,...,1999,2,2029,1,0,0,2.916667,1,0,30
4,711.841009,0,35644,0,1,0,78,18.0,109000,78.0,...,1999,2,2029,1,0,0,4.500000,2,0,30


In [4]:
df_raw.head()

,FirstPaymentDate,FirstTimeHomebuyer,MaturityDate,MSA,MIP,Units,Occupancy,OCLTV,DTI,OrigUPB,...,PostalCode,LoanSeqNum,LoanPurpose,OrigLoanTerm,NumBorrowers,SellerName,ServicerName,EverDelinquent,MonthsDelinquent,MonthsInRepayment
CreditScore,,,,,,,,,,,,,,,,,,,,,
0,199902,N,202901,16974,25,1,O,89,27,117000,...,60400,F199Q1268030,P,360,2,FL,WASHINGTONMUTUALBANK,0,0,52
0,199902,N,202901,19740,0,1,O,73,17,109000,...,80200,F199Q1015092,N,360,1,FT,CHASEHOMEFINANCELLC,0,0,144
0,199902,N,202901,29940,0,1,O,75,16,88000,...,66000,F199Q1266886,N,360,2,FL,WASHINGTONMUTUALBANK,0,0,67
0,199902,N,202901,31084,0,1,O,76,14,160000,...,90700,F199Q1178167,N,360,2,GM,GMACMTGECORP,0,0,35
0,199902,N,202901,35644,0,1,O,78,18,109000,...,7600,F199Q1178517,N,360,2,GM,GMACMTGECORP,0,0,54


In [5]:
def cal_emi(p,r,n):
    return p * r * (1 / (1 - (1 + r) ** (-n)))

df['EMI']=np.vectorize(cal_emi)(df["OrigUPB"], df["OrigInterestRate"] , df["OrigLoanTerm"])
df['EMI'].head()

0     789750.0
1     708500.0
2     605000.0
3    1100000.0
4     776625.0
Name: EMI, dtype: float64

In [6]:
df["totalPayment"]    = df["EMI"] * df["OrigLoanTerm"]
df["interestAmount"]  = df["totalPayment"] - df["OrigUPB"]
df["monthlyInterest"] = df["interestAmount"] / df["OrigLoanTerm"]

In [7]:
def principal(r, amount, emi, month):
    for i in range(month):
        interest = r * amount
        p = emi - interest
        amount -= p
    return amount

df["currentPrincipal"] = np.vectorize(principal)(df["OrigInterestRate"], df["OrigUPB"], df["EMI"], df["MonthsInRepayment"])
print(df["currentPrincipal"].head())

0    117000.0
1    109000.0
2     88000.0
3    160000.0
4    109000.0
Name: currentPrincipal, dtype: float64


In [8]:
def compute_monthly_income(dti, emi):
    if dti == 0:
        return 0.01
    return  emi / dti

df["monthlyIncome"] = np.vectorize(compute_monthly_income)(df["DTI"], df["EMI"])
print(df["monthlyIncome"].head())

0    29250.000000
1    41676.470588
2    37812.500000
3    78571.428571
4    43145.833333
Name: monthlyIncome, dtype: float64


In [9]:
def compute_pre_payment(dti, income):
    if dti < 40:
        return income/2
    else:
        return income * (3/4)

df["prePayment"] = np.vectorize(compute_pre_payment)(df["DTI"], df["monthlyIncome"] * 24)

M = max(df["prePayment"])
m = min(df["prePayment"])
print(f"max : {M}")
print(f"min : {m}")

max : 22680000.0
min : 21710.526315789473


In [10]:
df["SPP"] = (df["EMI"]-df["monthlyInterest"]) * df["OrigLoanTerm"]
df["APP"] = (df["EMI"]-df["monthlyInterest"]) * df["MonthsInRepayment"] + df["prePayment"]
df["OPB"] = df["currentPrincipal"]

df["PPR"] = abs(df["SPP"] - df["APP"]) / df["OPB"]

features = ["SPP", "APP", "OPB", "PRR"]
M = max(df["PPR"])
m = min(df["PPR"])

print(f"max : {M}")
print(f"min : {m}")

max : 98.37500000000003
min : 0.6443404837973886


In [11]:
data=df.copy()

In [12]:
X = data[[  'FirstPaymentMonth', 'IsFirstTimeHomeBuyer', 'Maturity_Month', 'MIP', 'Occupancy',
            'DTI', 'OrigUPB', 'OrigInterestRate', 'Channel', 'PPM',
            'PropertyState', 'PropertyType', 'LoanPurpose',
            'OrigLoanTerm', 'NumBorrowers', 'MonthsDelinquent',
            'Credit_range', 'LTV_range', 'Repay_range'
            ]]
y = data["PPR"]

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

In [14]:
cat_cols = X.select_dtypes("object").columns
num_cols = X.select_dtypes(exclude="object").columns
target = 'PPR'


categorical_pipeline = Pipeline(
    steps=[
        ("impute", SimpleImputer(strategy="constant")),
        ("one_hot_encode", OneHotEncoder(handle_unknown="ignore")),
    ]
)

numeric_pipeline = Pipeline(
    steps=[
        ("impute", SimpleImputer(strategy="mean"))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_pipeline, num_cols),
        ("categorical", categorical_pipeline, cat_cols),
    ]
)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, fbeta_score


In [18]:
X_train.shape

(233159, 19)

In [19]:
DT_model = DecisionTreeRegressor(criterion='squared_error')
DT_model.fit(X_train, y_train)
DT_pred = DT_model.predict(X_test)

DT_MSE = mean_squared_error(y_test, DT_pred)
DT_R2 = r2_score(y_test, DT_pred)

In [20]:
print(f"MSE ------>: {DT_MSE}")
print(f"R2 score ------->: {DT_R2}")

MSE ------>: 0.00286798594985073
R2 score ------->: 0.9994120542321232


In [21]:
XGB_model = XGBRegressor( max_depth=7, n_estimators=300, random_state=40 )
XGB_model.fit(X_train, y_train)
XGB_pred = XGB_model.predict(X_test)

XGB_MSE = mean_squared_error(y_test, XGB_pred)
XGB_R2 = r2_score(y_test, XGB_pred)

In [22]:
print(f"MSE ------>: {XGB_MSE}")
print(f"R2 score ------->: {XGB_R2}")

MSE ------>: 0.0013363518250694877
R2 score ------->: 0.9997260438462102


In [23]:
import pickle
file='MBSRiskshield_trained_model.sav'
pickle.dump(XGB_model,open(file,'wb'))

In [24]:
loaded_model=pickle.load(open('MBSRiskshield_trained_model.sav','rb'))
t=loaded_model.predict(X_test)
error_score = r2_score(y_test, t)
print("R squared Error : ", error_score)
# XGB_model.predict([1,'n',1,1,'o',0,100000,1,'T','n','il','sf','p',100,1,0,1,1,1])

R squared Error :  0.9997260438462102


In [25]:
user_Ip_cols = [  'FirstPaymentMonth', 'IsFirstTimeHomeBuyer', 'Maturity_Month', 'MIP', 'Occupancy',
            'DTI', 'OrigUPB', 'OrigInterestRate', 'Channel', 'PPM',
            'PropertyState', 'PropertyType', 'LoanPurpose',
            'OrigLoanTerm', 'NumBorrowers', 'MonthsDelinquent',
            'Credit_range', 'LTV_range', 'Repay_range'
            ]

In [26]:
for col in user_Ip_cols : 
    print(f"{col} : " ,df[col].unique())
    print("\n")
    print("_"*100)
    print("\n")

FirstPaymentMonth :  [ 2  3  4  5  6  7  8  9 10 11 12  1]


____________________________________________________________________________________________________


IsFirstTimeHomeBuyer :  [0 1]


____________________________________________________________________________________________________


Maturity_Month :  [ 1 10  2  9  3  7  4  5  6  8 11 12]


____________________________________________________________________________________________________


MIP :  [25  0 30 12 35 29 17 18 20 22  6 14 33 16 52 36 28 11 45 15 27 10 32  9
 50 47  7 24 37 26  1 13 21 19 23 55  4]


____________________________________________________________________________________________________


Occupancy :  [0 1 2]


____________________________________________________________________________________________________


DTI :  [27.         17.         16.         14.         18.         40.
 21.         20.         38.         19.         23.         45.
 28.         29.         15.         43.         25

In [30]:
from sklearn.decomposition import PCA
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=15)),
        ("model", XGBRegressor()),
    ]
)

In [31]:
pipeline.fit(X_train, y_train)
accuracy = pipeline.score(X_test, y_test)
print("Accuracy of the XGBoost model:", accuracy)

Accuracy of the XGBoost model: 0.6731859800086508


In [32]:
pickle.dump(pipeline, open("XGBreg_vasu_pipeline.pkl", "wb"))